# Automated App Deployment with CML APIv2
Use this Notebook after you have explored Bedrock, Pinecone and CrewAI to deploy your Chat app in Cloudera AI

#### 4.1 Import variables

In [1]:
import os
import cmlapi
import random
import string
import json

#### 4.2 Get CML API Client and list the available Runtimes
This code connects to your Cloudera Machine Learning (CML) environment, retrieves a list of available Python 3.10 runtimes with Nvidia GPU support and JupyterLab as the editor, prints the list, and then selects and stores the image identifier of the second runtime in the list. It also sets an environment variable APP_IMAGE_ML_RUNTIME with this image identifier for future use in launching jobs within the environment.

In [5]:
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
available_runtimes = client.list_runtimes(search_filter=json.dumps({
    "kernel": "Python 3.10",
    "edition": "Standard",
    "editor": "JupyterLab",
    "full_version": "2024.10.1-b12"
}))
print(available_runtimes)

# Set available runtimes to the latest runtime in the environment (iterator is the number that begins with 0 and advances sequentially)
# The JOB_IMAGE_ML_RUNTIME variable stores the ML Runtime which will be used to launch the job
print(available_runtimes.runtimes[0])
print(available_runtimes.runtimes[0].image_identifier)
APP_IMAGE_ML_RUNTIME = available_runtimes.runtimes[0].image_identifier

## Store the ML Runtime for any future jobs in an environment variable so we don't have to do this step again
os.environ['APP_IMAGE_ML_RUNTIME'] = APP_IMAGE_ML_RUNTIME

{'next_page_token': '',
 'runtimes': [{'description': 'Standard edition JupyterLab Python runtime '
                              'provided by Cloudera',
               'edition': 'Standard',
               'editor': 'JupyterLab',
               'full_version': '2024.10.1-b12',
               'image_identifier': 'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-standard:2024.10.1-b12',
               'kernel': 'Python 3.10',
               'register_user_id': 0,
               'status': 'ENABLED'}]}
{'description': 'Standard edition JupyterLab Python runtime provided by '
                'Cloudera',
 'edition': 'Standard',
 'editor': 'JupyterLab',
 'full_version': '2024.10.1-b12',
 'image_identifier': 'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-standard:2024.10.1-b12',
 'kernel': 'Python 3.10',
 'register_user_id': 0,
 'status': 'ENABLED'}
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-standa

#### 4.3 Get the current working project
Here we get the current project from the environment variable "CDSW Project ID" and print its metadata.

In [6]:
project = client.get_project(project_id=os.getenv("CDSW_PROJECT_ID"))
print(project)

{'created_at': datetime.datetime(2024, 11, 22, 23, 24, 27, 649274, tzinfo=tzlocal()),
 'creation_status': 'success',
 'creator': {'email': 'sbalachandar@cloudera.com',
             'name': 'Shreshta Balachandar',
             'username': 'sbalachandar'},
 'default_engine_type': 'ml_runtime',
 'description': '',
 'environment': '{"CDSW_APP_POLLING_ENDPOINT":"/","PROJECT_OWNER":"sbalachandar"}',
 'ephemeral_storage_limit': 30,
 'ephemeral_storage_request': 0,
 'id': 'xd1j-uzqr-043d-m60m',
 'name': 'Cloudera Agentic AI',
 'owner': {'email': 'sbalachandar@cloudera.com',
           'name': 'Shreshta Balachandar',
           'username': 'sbalachandar'},
 'permissions': {'admin': True,
                 'business_user': True,
                 'inherit': False,
                 'operator': True,
                 'read': True,
                 'write': True},
 'shared_memory_limit': 0,
 'updated_at': datetime.datetime(2024, 11, 25, 19, 58, 3, 17220, tzinfo=tzlocal()),
 'visibility': 'private'}


#### 4.4 Create and Run Application for Hosted LLM Application
This code creates a Cloudera Machine Learning (CML) application with the name "CML LLM Gradio Interface" and a description, associates it with a specific project (project.id), assigns it a subdomain, specifies Python 3 as the kernel, and provides a script path for the application. It also sets resource specifications for CPU and memory and assigns the runtime identifier obtained from the environment variable APP_IMAGE_ML_RUNTIME. Finally, it creates the application within the specified project using the client. 

In [7]:
# DO NOT EXECUTE UNTIL YOU SET A UNIQUE SUBDOMAIN
application_request = cmlapi.CreateApplicationRequest(
     name = "Agentic AI Chat Interface",
     description = "Hosted interface for the CML LLM Gradio UI",
     project_id = project.id,
     subdomain = "agentic-ai-chat-123", # THIS MUST BE UNIQUE PER USER
     script = "chat_app/agents_app_launch.py",
     cpu = 1,
     memory = 4,
     runtime_identifier = os.getenv('APP_IMAGE_ML_RUNTIME')
)

app = client.create_application(
     project_id = project.id,
     body = application_request
)